<a href="https://colab.research.google.com/github/HassanSherwani/Product_Purchase_Frequency/blob/master/Product_Freq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem Statement:

How often a certain product has been sold in the past days.

#1)- Importing key modules

In [0]:
# Let's be rebels and ignore warnings for now
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys

In [3]:
! pip install turicreate

In [0]:
import turicreate as tc

# 2)-Loading Data

In [0]:
url = 'https://raw.githubusercontent.com/HassanSherwani/Product_Purchase_Frequency/master/20190207_transactions%20.json'

In [0]:
transactions = pd.read_json(url, lines= True)

# 3)-Exploring dataset

In [7]:
transactions.head()

,id,products
0,0,"[185, 30, 77, 188, 78, 125, 45, 155, 241, 229,..."
1,1,"[119, 148, 108, 34, 157, 82, 113, 45, 165]"
2,2,"[173, 103, 229, 240]"
3,3,[91]
4,4,"[175, 192, 54, 172]"


In [8]:
transactions.shape

(2500, 2)

In [9]:
print(transactions['products'][1415])

[250, 236, 242, 229, 92, 2, 71, 172, 109, 247, 171, 209, 90, 139, 188, 191, 145, 214, 216, 237]


In [10]:
transactions.info() # checking missing values

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500 entries, 0 to 2499
Data columns (total 2 columns):
id          2500 non-null int64
products    2500 non-null object
dtypes: int64(1), object(1)
memory usage: 138.6+ KB


In [11]:
transactions.describe()

,id
count,2500.00000
mean,1249.50000
std,721.83216
min,0.00000
25%,624.75000
50%,1249.50000
75%,1874.25000
max,2499.00000


# 4)- Adding Features:

### 4.1)-Create a separate dataframe for recommending users

In [0]:
customers=transactions['id']

In [13]:
customers.head()

0    0
1    1
2    2
3    3
4    4
Name: id, dtype: int64

In [0]:
import random
random.shuffle(customers) 

 To get random values of customer ids. This would help us in avoiding sample biaseness in our modeling.

In [0]:
cust_2_rec=customers[:1000]

In [16]:
cust_2_rec.head()

0     845
1    2290
2    1881
3      23
4     303
Name: id, dtype: int64

### 4.2)- break down each list of items in the products column into rows and count the number of products bought by a user

In [17]:


pd.melt(transactions.head(2).set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})

,id,productId,purchase_count
0,845,30.0,1
1,845,45.0,1
2,845,77.0,1
3,845,78.0,1
4,845,89.0,2
5,845,125.0,1
6,845,133.0,1
7,845,155.0,1
8,845,161.0,1
9,845,185.0,1


# 5- Creating features for user_id,product_id and purchase count

### 5a)-One for purchase count

In [18]:
s=time.time()

data = pd.melt(transactions.set_index('id')['products'].apply(pd.Series).reset_index(), 
             id_vars=['id'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['id', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})
data['productId'] = data['productId'].astype(np.int64)

print("Execution time:", round((time.time()-s)/60,2), "minutes")

Execution time: 0.01 minutes


In [19]:
data.shape

(24811, 3)

In [20]:
data.head()

,id,productId,purchase_count
0,0,32,1
1,0,47,1
2,0,114,1
3,0,128,1
4,0,140,1


### 5-b)-Dummy as target

Dummy for marking an assumption whether a customer bought that item or not.
If one buys an item, then purchase_dummy are marked as 1.

In [0]:
 
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy

In [0]:
data_dummy = create_data_dummy(data)

In [23]:
data_dummy.head()

,id,productId,purchase_count,purchase_dummy
0,0,32,1,1
1,0,47,1,1
2,0,114,1,1
3,0,128,1,1
4,0,140,1,1


### 5-c)-Normalize item

we normalize purchase frequency of each item across users by first creating a user-item matrix 

In [0]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')

In [25]:
df_matrix.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0
3,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The NaN tells us that the item represented by the column was not purchased in that specific transaction.

In [26]:
df_matrix.shape

(2378, 250)

In [27]:
df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())

df_matrix_norm.head()

productId,1,2,3,4,5,6,7,8,9,10,...,241,242,243,244,245,246,247,248,249,250
id,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0
3,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
print(df_matrix_norm.shape)

(2378, 250)


### 5-d)- create a table for input to the modeling

In [0]:
d = df_matrix_norm.reset_index()
d.index.names = ['scaled_purchase_freq']
data_norm = pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()

In [30]:
data_norm.head()

,id,productId,scaled_purchase_freq
1,1,1,0.0
4,4,1,0.0
11,12,1,0.0
18,21,1,0.0
37,42,1,0.0


In [31]:
data_norm.shape

(22530, 3)

### 5-e) a function for normalizing data

In [0]:
def normalize_data(data):
    df_matrix = pd.pivot_table(data, values='purchase_count', index='id', columns='productId')
    df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
    d = df_matrix_norm.reset_index()
    d.index.names = ['scaled_purchase_freq']
    return pd.melt(d, id_vars=['id'], value_name='scaled_purchase_freq').dropna()


we have normalized the items accoreding to their purchase history, from 0–1 (with 1 being the most number of purchase for an item and 0 being 0 purchase count for that item.

# 6)-Split train and test set

we have three dataframes with purchase counts(data), purchase dummy(data_dummy), and scaled purchase counts(data_norm).

In [33]:
train, test = train_test_split(data, test_size = .2)
print(train.shape, test.shape)

(19848, 3) (4963, 3)


In [0]:
#convert dataframe to SFrame
train_data = tc.SFrame(train)
test_data = tc.SFrame(test)

A tabular, column-mutable dataframe object that can scale to big data <br>
https://turi.com/products/create/docs/generated/graphlab.SFrame.html

In [35]:
train_data

id,productId,purchase_count
687,2,1
1118,145,1
1033,205,1
714,119,1
949,142,1
750,87,1
124,176,1
299,225,1
2223,65,1
367,46,1


In [36]:
test_data

id,productId,purchase_count
645,7,1
1567,144,1
2336,223,2
169,231,1
2178,56,1
1788,50,1
2114,134,1
2057,22,1
479,46,1
416,178,1


### 6.2)- Define a split_data function 

In [0]:
def split_data(data):

    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data

### 6.3)-Apply for both dummy table and scaled/normalized purchase table

In [0]:
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

# 7)-Building Model

### 7.1)- Parameters to define field names for purchase count as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_count'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [40]:
popularity_model = tc.popularity_recommender.create(train_data, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 19848 observations with 2348 users and 250 items.

Data prepared in: 0.036176s

19848 observations to process; with 250 unique items.

In [41]:
popularity_recomm = popularity_model.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 104657

+------+-----------+--------------------+------+
|  id  | productId |       score        | rank |
+------+-----------+--------------------+------+
| 845  |     96    | 1.0843373493975903 |  1   |
| 845  |    153    | 1.0759493670886076 |  2   |
| 845  |    207    | 1.0731707317073171 |  3   |
| 845  |    112    | 1.0714285714285714 |  4   |
| 845  |    201    | 1.0694444444444444 |  5   |
| 845  |    101    | 1.0684931506849316 |  6   |
| 845  |    125    | 1.0666666666666667 |  7   |
| 845  |    131    | 1.0649350649350648 |  8   |
| 845  |     44    | 1.064516129032258  |  9   |
| 845  |    152    | 1.063157894736842  |  10  |
| 2290 |     96    | 1.0843373493975903 |  1   |
| 2290 |    153    | 1.0759493670886076 |  2   |
| 2290 |    207    | 1.0731707317073171 |  3   |
| 2290 |    112    | 1.0714285714285714 |  4   |
| 2290 |    201    | 1.0694444444444444 |  5   |
| 2290 |    101    | 1.0684931506849316 |  6   |
| 2290 |    125    | 1.0666666666666667 |  7   |
| 2290 |    131    |

### Note 

Through this model, we predicted the recommendation items using scores by popularity. As you can tell for each model results above, the rows show the first 30 records from 1000 users with 10 recommendations.<br> These 30 records include 3 users and their recommended items, along with score and descending ranks.

## Checking most frequent items(as per assignment) 

In [42]:
 train.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.084337
153    1.075949
207    1.073171
112    1.071429
201    1.069444
101    1.068493
125    1.066667
131    1.064935
44     1.064516
63     1.063158
152    1.063158
186    1.058824
120    1.058140
102    1.057471
150    1.056818
49     1.056338
26     1.054795
35     1.054054
247    1.054054
232    1.054054
Name: purchase_count, dtype: float64

**products 96,125 201, 153, and 152 are the most popular (best-selling) across customers.**

### 7.2)- purchase dummy as target feature

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'purchase_dummy'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [44]:
popularity_model_dummy = tc.popularity_recommender.create(train_data_dummy, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 19848 observations with 2355 users and 250 items.

Data prepared in: 0.041342s

19848 observations to process; with 250 unique items.

In [45]:
popularity_recomm_4_dummy = popularity_model_dummy.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_dummy.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 129282

+------+-----------+-------+------+
|  id  | productId | score | rank |
+------+-----------+-------+------+
| 845  |    241    |  1.0  |  1   |
| 845  |    138    |  1.0  |  2   |
| 845  |     83    |  1.0  |  3   |
| 845  |     94    |  1.0  |  4   |
| 845  |     82    |  1.0  |  5   |
| 845  |     55    |  1.0  |  6   |
| 845  |    127    |  1.0  |  7   |
| 845  |     3     |  1.0  |  8   |
| 845  |    137    |  1.0  |  9   |
| 845  |    220    |  1.0  |  10  |
| 2290 |    241    |  1.0  |  1   |
| 2290 |    138    |  1.0  |  2   |
| 2290 |     83    |  1.0  |  3   |
| 2290 |     30    |  1.0  |  4   |
| 2290 |     94    |  1.0  |  5   |
| 2290 |     55    |  1.0  |  6   |
| 2290 |    127    |  1.0  |  7   |
| 2290 |     3     |  1.0  |  8   |
| 2290 |    137    |  1.0  |  9   |
| 2290 |    220    |  1.0  |  10  |
| 1881 |    138    |  1.0  |  1   |
| 1881 |     83    |  1.0  |  2   |
| 1881 |     30    |  1.0  |  3   |
| 1881 |     94    |  1.0  |  4   |
| 1881 |     82    |  1.0  |

### 7.3)- Applying 'scaled_purchase_freq' as target feature on model

In [0]:
user_id = 'id'
item_id = 'productId'
users_to_recommend=list(cust_2_rec)
target = 'scaled_purchase_freq'
n_rec = 10 # number of items to recommend
n_display = 30 # 1st 30 rows to display

In [47]:
popularity_model_scaled = tc.popularity_recommender.create(train_data_norm, 
                                                    user_id=user_id, 
                                                    item_id=item_id, 
                                                    target=target)

Preparing data set.

Data has 18024 observations with 2337 users and 227 items.

Data prepared in: 0.032925s

18024 observations to process; with 227 unique items.

In [48]:
popularity_recomm_4_scaled = popularity_model_scaled.recommend(users=users_to_recommend, k=n_rec)
popularity_recomm_4_scaled.print_rows(n_display)

recommendations finished on 1000/1000 queries. users per second: 124146

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 845  |    153    | 0.07792207792207792  |  1   |
| 845  |    129    | 0.07246376811594203  |  2   |
| 845  |    201    | 0.07142857142857142  |  3   |
| 845  |     96    | 0.06666666666666667  |  4   |
| 845  |    150    | 0.06382978723404255  |  5   |
| 845  |    156    | 0.06329113924050633  |  6   |
| 845  |     44    | 0.06097560975609756  |  7   |
| 845  |    207    | 0.060240963855421686 |  8   |
| 845  |    166    | 0.05813953488372093  |  9   |
| 845  |    186    | 0.056338028169014086 |  10  |
| 2290 |    153    | 0.07792207792207792  |  1   |
| 2290 |    129    | 0.07246376811594203  |  2   |
| 2290 |    201    | 0.07142857142857142  |  3   |
| 2290 |     96    | 0.06666666666666667  |  4   |
| 2290 |    150    | 0.06382978723404255  |  5   |
| 2290 |    156    | 0.06329113924050633  |  6   |
| 2290 |     44    | 0.06097560

Great!.... We have all three models worked out. We have purchase frequency of each item customer-vice. <br>
Which one to trust and which one to discard? 

# 8)- Evaluate models

### 8.1)- Validation for Popularity Model on Purchase Counts

In [0]:
models_counts = [popularity_model]

In [0]:
model_names=['Popularity Model on Purchase Counts']

In [51]:
eval_counts = tc.recommender.util.compare_models(test_data, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Purchase Counts


recommendations finished on 1000/1927 queries. users per second: 54570.3


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.008822003113648173 | 0.0028072256406454672 |
|   2    | 0.009340944473274541 |  0.006969464831426454 |
|   3    | 0.010032866286109672 |  0.011579393909440624 |
|   4    | 0.010119356512714067 |  0.015247815092132702 |
|   5    | 0.00934094447327453  |  0.01724635711402708  |
|   6    |  0.0096869053796921  |  0.02188223326002254  |
|   7    | 0.009415078953221146 |  0.024347204718247778 |
|   8    | 0.01018422418266737  |   0.0305448472417855  |
|   9    | 0.01014818658824885  |  0.034367715257699716 |
|   10   | 0.009963674104826162 |  0.03779334601856654  |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.1508992299177829

Per User RMSE (best)
+-----+------+-------+
|  id | rmse | count |
+-----+-----

### 8.2)- Validation of Popularity Model on Purchase Counts (dummy)

In [52]:
models_counts = [popularity_model_dummy]
model_names=['Popularity Model on Dummy Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_dummy, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Dummy Purchase Counts


recommendations finished on 1000/1912 queries. users per second: 55944.1


Precision and recall summary statistics by cutoff
+--------+----------------------+-----------------------+
| cutoff |    mean_precision    |      mean_recall      |
+--------+----------------------+-----------------------+
|   1    | 0.009937238493723846 | 0.0037507471607890006 |
|   2    | 0.010460251046025128 |  0.007826509264793783 |
|   3    | 0.011331938633193862 |  0.012102136879856552 |
|   4    | 0.010721757322175722 |  0.015254532775453281 |
|   5    | 0.011087866108786607 |  0.021253196187819578 |
|   6    | 0.010373082287308218 |  0.023580602045560218 |
|   7    | 0.010236102809324542 |  0.028104660622966054 |
|   8    |  0.0103294979079498  |  0.03294750780367942  |
|   9    | 0.010634588563458863 |  0.03890113402404216  |
|   10   | 0.010407949790794997 |  0.041978191206747714 |
+--------+----------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+-----+------+-------+
|  id | rmse | count |
+-----+------+-------+
| 91

### 8.3)- Validation for Popularity Model on Scaled Purchase Counts

In [53]:
models_counts = [popularity_model_scaled]
model_names=['Popularity Model on Scaled Purchase Counts']
eval_counts_dummy = tc.recommender.util.compare_models(test_data_norm, models_counts, model_names)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts


recommendations finished on 1000/1838 queries. users per second: 54194.7


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.011969532100108796 | 0.004824084149437803 |
|   2    | 0.01278563656147988  | 0.010288097828903086 |
|   3    | 0.011425462459194779 | 0.013869889631587135 |
|   4    | 0.012105549510337326 | 0.019902153824895897 |
|   5    | 0.011969532100108824 | 0.02465369535554519  |
|   6    | 0.011969532100108806 | 0.02937803340414878  |
|   7    | 0.012435877506606562 | 0.035830440264607855 |
|   8    | 0.012649619151251373 | 0.04203801578665563  |
|   9    | 0.012453149558699075 | 0.04622735202169369  |
|   10   | 0.012459194776931432 | 0.053454410418501866 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 0.16214960859529115

Per User RMSE (best)
+------+------+-------+
|  id  | rmse | count |
+------+------+-------+

Dummy purchase count model gives lowest rmse . So we will choose that as best model for our final step.

# 9) Submission

In [54]:
users_to_recommend = list(cust_2_rec)

final_model = tc.item_similarity_recommender.create(tc.SFrame(data_dummy), 
                                            user_id=user_id, 
                                            item_id=item_id, 
                                            target='purchase_dummy')


Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 24811 observations with 2378 users and 250 items.

Data prepared in: 0.042781s

...

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 4.49ms                              | 0                | 0               |

| 29.569ms                            | 100              | 250             |

+-------------------------------------+------------------+-----------------+

In [55]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 1.034s

recommendations finished on 1000/1000 queries. users per second: 42970.1

+------+-----------+----------------------+------+
|  id  | productId |        score         | rank |
+------+-----------+----------------------+------+
| 845  |    248    | 0.025486195087432863 |  1   |
| 845  |     23    | 0.024235220750172932 |  2   |
| 845  |     63    | 0.024063706398010254 |  3   |
| 845  |    168    |  0.0234219233194987  |  4   |
| 845  |     32    | 0.02222420374552409  |  5   |
| 845  |    110    | 0.021273497740427652 |  6   |
| 845  |    214    | 0.021062183380126952 |  7   |
| 845  |    126    | 0.02020918925603231  |  8   |
| 845  |     74    | 0.01874724229176839  |  9   |
| 845  |     6     | 0.017863396803538004 |  10  |
| 2290 |     23    | 0.027408692571851943 |  1   |
| 2290 |     17    | 0.027234587404463027 |  2   |
| 2290 |    177    | 0.026982453134324815 |  3   |
| 2290 |     63    | 0.025824248790740967 |  4   |
| 2290 |    214    | 0.025738835334777832 |  5   |
| 2290 |     80    | 0.02526219023598565  |  6   |
| 2290 |    112    | 0.02509811

### 9.2)- Checking most frequent items in final model

In [56]:
data_dummy.groupby(by=item_id)['purchase_count'].mean().sort_values(ascending=False).head(20)

productId
96     1.080808
207    1.076923
153    1.071429
201    1.069767
26     1.063158
152    1.060345
125    1.058824
112    1.058824
85     1.058824
131    1.057143
35     1.056180
63     1.054545
44     1.054545
129    1.053763
101    1.053763
117    1.052632
150    1.051724
122    1.051724
156    1.051546
120    1.050000
Name: purchase_count, dtype: float64

**RESULT: <BR>
products 96,207 , 153,201 and 26(and so on) are the most frequent selling items across customers.**

# 10)- Bonus Part- Recommending products to customers

In [57]:
# Create a csv output file
df_rec = recom.to_dataframe()
df_rec.head()

,id,productId,score,rank
0,845,248,0.025486,1
1,845,23,0.024235,2
2,845,63,0.024064,3
3,845,168,0.023422,4
4,845,32,0.022224,5


In [58]:
print(df_rec.shape)

(10000, 4)


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([id])[item_id].transform(lambda x: '|'.join(x.astype(str)))
df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates().sort_values('id').set_index('id')

In [60]:
recomendation = final_model.recommend(users=users_to_recommend, k=n_rec)

recommendations finished on 1000/1000 queries. users per second: 44033.5

In [0]:
df_rec = recomendation.to_dataframe()


In [0]:
df_rec['recommendedProducts'] = df_rec.groupby([user_id])[item_id] \
        .transform(lambda x: '|'.join(x.astype(str)))

In [0]:

df_output = df_rec[['id', 'recommendedProducts']].drop_duplicates() \
        .sort_values('id').set_index('id')

In [64]:
df_output.head()

,recommendedProducts
id,
0,23|211|54|100|195|45|36|133|25|152
1,169|246|45|74|153|83|30|204|150|85
3,173|23|17|122|152|166|227|54|40|69
4,41|38|77|202|36|11|69|13|147|61
7,250|36|2|234|50|152|153|235|55|77
